# Macroscopic Fundamental Diagrams
## CEE 498- Transportation Economics

Noah Garfinkle

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import ortools as ort